In [1]:
import sys
import pandas as pd

sys.path.append('..')
from etl import create_spark_session

In [2]:
spark = create_spark_session(local=True)
spark.sparkContext.setLogLevel("ERROR")
spark

In [3]:
spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/songplays.parquet').registerTempTable('songplays')
spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/times.parquet').registerTempTable('times')
spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/artists.parquet').registerTempTable('artists')
spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/users.parquet').registerTempTable('users')
spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/songs.parquet').registerTempTable('songs')

In [9]:
#TODO: Check why there of weekday as NAN
spark.sql("""
SELECT songplays.level, times.weekday, artists.name, COUNT(*) as count
FROM songplays
JOIN times on songplays.start_time = times.start_time
JOIN artists on songplays.artist_id = artists.artist_id
GROUP BY CUBE (songplays.level, times.weekday, artists.name)
ORDER BY count DESC, weekday ASC
LIMIT 10
""").write.format('parquet').mode('overwrite').save('s3a://dutrajardim/olap-cubes/level-weekday-artists.parquet')

In [12]:
spark.sql("""
SELECT DISTINCT(weekday)
FROM times
""").show()

+-------+
|weekday|
+-------+
|      1|
|      6|
|      3|
|      5|
|      4|
|      7|
|      2|
+-------+



In [13]:
spark.stop()